In [ ]:
# import talib

# talib.get_function_groups()['Overlap Studies']

# ?talib.MAVP

# dir(talib)

# print(talib.get_functions())
# print(len(talib.get_functions()))


# list(talib.get_function_groups().keys())

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)



In [ ]:
import talib
import pandas as pd
from pandas import Series, DataFrame

df = pd.read_csv('C:\\Users\DELL\Desktop\CryptoData\SOL110820_170522_1h.csv', index_col=0)
print(df.shape) #total rows available
df = df.iloc[-1500:,:]
print(df.shape) #rows we are using
# print(df)

from talib import abstract #change columnn names to lower-case
df.rename(columns={'Open':'open', 'High':'high','Low':'low','Close':'close','Volume':'volume'},inplace=True)

f = abstract

In [ ]:

#candlestick patterns -+200:Bullish with confirm ; +100:Bullish ; 0:None ; -100:Sell ; -200:Bearish with confirm
dir_candle = [candle for candle in dir(f) if candle.startswith('CDL')]

#cycle indicators
dir_cycle = list(talib.get_function_groups()['Cycle Indicators'])

#math operators
dir_moperators = list(talib.get_function_groups()['Math Operators'])

#math transform
dir_mtransform = list(talib.get_function_groups()['Math Transform'])

#momentum indicators
dir_momentum = list(talib.get_function_groups()['Momentum Indicators'])

#overlap studies
dir_overlap = list(talib.get_function_groups()['Overlap Studies'])
dir_overlap.remove('MAVP') #no use known as manual parameter 'periods' required

#price transform
dir_ptransform = list(talib.get_function_groups()['Price Transform'])

#statistic functions
dir_statistics = list(talib.get_function_groups()['Statistic Functions'])


#volatility indicators
dir_volatility = list(talib.get_function_groups()['Volatility Indicators'])

#volume indicators
dir_volume = list(talib.get_function_groups()['Volume Indicators'])


#everything
all_dir = list(talib.get_functions())

# whichever to be added to dataframe
my_indi =  dir_cycle + dir_moperators + dir_mtransform + dir_momentum + dir_overlap + dir_ptransform \
+ dir_statistics + dir_volatility + dir_volume # dir_candle +

#adding all patterns and indicators to the dataframe
for indicator in my_indi:
    x =str(indicator)
    y = getattr(f,x)(df)

    if type(y) == Series:
        df[indicator] = y.tolist()
    
    elif type(y) == DataFrame:
        for in_indi in y:
            df[in_indi] = y[in_indi]

# df

In [ ]:
# copy of dataframe with everything
# df_full = df.copy()
# df_full.shape
# df_full
df

In [ ]:
#creating target column with percentage change in close
df['close_delta'] = df['close'].pct_change().shift(periods=-1)

#dropping opentime and closetime as of no use for now
df = df.drop(['open_time', 'close_time', 'qav', 'num_trades', 'taker_base_vol', 'taker_quote_vol', 'ignore'], axis=1)

# dropping columns with all values as NaN
df.dropna(axis=1, how='all', inplace=True)
# df.drop(['Unnamed: 0'], axis=1, inplace=True)

#dropping rows with any values as NaN
df.dropna(inplace=True)

print(df.shape)
df

In [ ]:
# # just for checking
# df_half = df.copy()
# df_half

# col1 = list(df_full.columns)

# #for checking which columns contain only NaN values
# for aam in col1:
#     if aam not in df_half.columns:
#         print(aam)
#         print(df_full.columns.get_loc(aam))

In [ ]:
df['ADirection'] = [1 if i > 0.005 else i for i in df['close_delta']]
df['BDirection'] = [0.5 if 0.005 > i > 0  else i for i in df['ADirection']]
df['CDirection'] = [-0.5 if -0.005 < i < 0  else i for i in df['BDirection']]
df['Direction'] = [-1 if -0.005 > i else i for i in df['CDirection']]

df = df.drop(['ADirection', 'BDirection', 'CDirection', 'close_delta'], axis=1)


df

In [ ]:
df.tail()

In [ ]:
import tensorflow as tf 
print(tf.__version__)

In [ ]:

# independant variables
X = df.drop(['Direction'], axis=1)

# the dependent variable
Y = df[['Direction']]

In [ ]:

print(X.shape, Y.shape)

In [ ]:
# Split X and y into training and test set in 70:30 ratio

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=1, shuffle=True)

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

In [ ]:
from scipy.stats import zscore
X =X.apply(zscore)
X.head()

In [ ]:
X_train[0:1]  # let's have a look at one row

Y_train[:-10]  # let's have a look at the first 10 training labels

In [ ]:

X_train = np.array(X_train)
print(X_train.shape)
X_train = X_train[:, tf.newaxis, :]
print(X_train.shape)

X_test = np.array(X_test)
print(X_test.shape)
X_test = X_test[:, tf.newaxis, :]
print(X_test.shape)

In [ ]:
model = keras.Sequential([
    keras.layers.Input(shape=(1, 5)),  # input layer (1)
    keras.layers.Dense(4, activation='elu'),  # hidden layer (2)
    keras.layers.Dense(4, activation='softmax') # output layer (3)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, batch_size=10, epochs=10 )  # we pass the data, labels and epochs and watch the magic!

In [ ]:
test_loss, test_acc = model.evaluate(X_test,  Y_test, verbose=1) 

print('Test accuracy:', test_acc)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions[0]

In [ ]:
np.argmax(predictions[0])

In [ ]:
Y_test[0]


In [8]:
import tensorflow as tf

In [9]:
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
